In [ ]:
import os

%load_ext autoreload
%autoreload 2

img_dir = "outputs/imgs/weekdays/"
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [ ]:
"""
Read and clean the meal entry data

"""

from analysis_utils import clean

meal_info = clean.cleaned_smartwatch(keep_catchups=True, keep_day0=False)

meal_info = meal_info[["week_day", "p_id", "delta"]]

meal_info["entry_day"] = meal_info["delta"].dt.days
meal_info.drop(columns=["delta"], inplace=True)
meal_info.head(5)

In [ ]:
"""
Keep only the weekday where the entry day is 1- i.e. the start day of the study

"""
import numpy as np

start_days = []
for (p_id, day), group in meal_info.groupby(["p_id", "entry_day"]):
    if day != 1:
        continue

    # Get the start day, ensuring that there is only one unique start day for this p_id
    # on day 1
    start_day, = group["week_day"].unique()

    start_days.append(start_day)

day_counts = dict(zip(*np.unique(start_days, return_counts=True)))
print(day_counts)

In [ ]:
""" Sort the days """

natural_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sorted_day_counts = {day: day_counts.get(day, 0) for day in natural_order if day in day_counts}

sorted_day_counts

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(sorted_day_counts.keys(), sorted_day_counts.values())
plt.title("Study Start Day")
plt.savefig(img_dir + "study_start_day.png")